In [2]:
import os,sys
from pathlib import Path
from typing import Any,List, Set, Dict, Tuple, Optional, Iterable, Mapping, Union, Callable, TypeVar, cast

import time
from datetime import datetime
from collections import OrderedDict
from functools import partial

# data or cache IO
import json
import joblib

# print, debug
from pprint import pprint
from IPython.display import display, HTML # for pretty-print pandas dataframe
from IPython.core.debugger import set_trace

sys.dont_write_bytecode = True

# numpy and friends
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


# # Select Visible GPU
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
# DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Reprlearn
from reprlearn.visualize.utils import get_fig, show_timg, show_timgs, show_npimgs, show_batch 
from reprlearn.utils.misc import info, now2str, get_first_img_info, count_imgs
from reprlearn.utils.misc import is_valid_dir, is_img_fp





## Simple rename (move) operation

In [2]:
# move files to a target dir
# from pathlib import Path

# root = Path("/path/to/your/root/dir")

# for file in root.glob("*.pdf"):
#     folder_name = file.stem.rpartition("_")[-1]
    # file.rename(root / folder_name / file.name)


root = Path("xxx")
model_name = 'vae-nvae'
target_dir = root/model_name
assert target_dir.exists()




In [8]:
skipped = [] 
n_done = 0
for fp in root.iterdir():
    if fp.is_dir() or not fp.suffix.lower() == '.png':
        skipped.append(fp)
        continue
    # print(fp)
    new_fp = target_dir / fp.name 
    fp.rename(new_fp)
    # print(new_fp)
    n_done += 1
    

print(f'Done renaming: {n_done} files')
        

Done renaming: 49998 files


## Add a prefix to each images in a folder
- date: 20230911-171948
- usage: use this script to make each sampling run's output image's filenames unique across different runs, by adding each run's unique run_id to the files genearted in that run
- Log:
  - i use it before collagating all image files genearted by 3 different sampling runs of `score-ldm` model
  

In [21]:
img_dir = Path("xxx")
run_id = ''.join(img_dir.parent.stem.split('-'))
print(f'run_id: {run_id}')

run_id: 20230828172610


In [24]:
# add this run_id to each img file's name
n_done = 0
prefix = run_id
for fp in img_dir.iterdir():
    if not is_img_fp(fp):
        continue
    # if n_done > 10: #debug
    #     break 
    new_name = f"{prefix}_{fp.name}"
    new_fp = fp.parent / new_name
    # print(fp)
    # print(new_fp)
    # print('')
    
    #===!!! do rename operation:
    fp.rename(new_fp)
    n_done += 1
    
print(f'Done: {n_done} files renamed')
    

Done: 3630 files renamed


In [36]:
# todo: incorporate this to reprlearn library
def add_prefix_to_each_filename(img_dir: Path,
                                prefix: str,
                                is_valid_fp: Optional[Callable]=None,
                                is_dryrun: Optional[bool]=False,
                                debug: Optional[bool]=False,
                               )-> None:
    is_valid_fp = is_valid_fp or is_img_fp
    
    n_done = 0
    for fp in img_dir.iterdir():
        if not is_valid_fp(fp):
            continue

        new_name = f"{prefix}_{fp.name}"
        new_fp = fp.parent / new_name
        
        if debug:
            print(f'\noriginal fp: {fp}')
            print(f'new fp: {new_fp}')
            if n_done >= 10:
                break 

        #===!!! do rename operation:
        if not is_dryrun:
            fp.rename(new_fp)
            pass
        n_done += 1

    print(f'Done: {n_done} files renamed') 

### Apply the prefix-adding to images in VQGAN-genearted images in the following two dirs:


#### First dir

In [37]:
img_dir = Path(
    "xxx"
)
prefix = "20230904-141326"

add_prefix_to_each_filename(
    img_dir=img_dir,
    prefix=prefix,
    is_valid_fp=is_img_fp,
    # is_dryrun=True,
    # debug=True
)


Done: 2870 files renamed


In [38]:
# check the prefix is appeneded
!ls {img_dir} | head 

20230904-141326_000000.png
20230904-141326_000001.png
20230904-141326_000002.png
20230904-141326_000003.png
20230904-141326_000004.png
20230904-141326_000005.png
20230904-141326_000006.png
20230904-141326_000007.png
20230904-141326_000008.png
20230904-141326_000009.png
ls: write error: Broken pipe
